<a href="https://colab.research.google.com/github/minjaekim070/Bootcamp_Project/blob/main/CNN%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_3%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

전체 데이터셋의 다운로드가 오래 걸려서 bottle 폴더에 대한 내용을 우선적으로 정상/비정상으로 분류할 수 있도록 진행

#Library import


In [1]:
# 패키지 import
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torchvision import transforms, datasets, utils
from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.preprocessing import image


#이미지 업로드 및 배열 변환


In [2]:
# 이미지 경로
train_data_dir = '/content/drive/MyDrive/bottle/train/good'
large_data_dir = '/content/drive/MyDrive/bottle/test/broken_large'
small_data_dir = '/content/drive/MyDrive/bottle/test/broken_small'
contamination_data_dir='/content/drive/MyDrive/bottle/test/contamination'
test_data_dir = '/content/drive/MyDrive/bottle/test/good'
gr_tr_large_dir = '/content/drive/MyDrive/bottle/ground_truth/broken_large'
gr_tr_small_dir = '/content/drive/MyDrive/bottle/ground_truth/broken_small'
gr_tr_contamination_dir = '/content/drive/MyDrive/bottle/ground_truth/contamination'

In [3]:
#이미지를 데이터로 변환
def load_image_data(image_dir):
    image_data = []
    for img_name in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_name)
        img = image.load_img(img_path, target_size=(64, 64))  # 이미지를 지정된 크기로 로드합니다
        img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
        image_data.append(img_array)
    return image_data

def load_data(data_dir, test_data_dir, large_data_dir, small_data_dir, contamination_data_dir):
    train_image_data = load_image_data(data_dir)
    test_image_data = load_image_data(test_data_dir)
    large_image_data = load_image_data(large_data_dir)
    small_image_data = load_image_data(small_data_dir)
    contamination_image_data = load_image_data(contamination_data_dir)
    return train_image_data, test_image_data, large_image_data, small_image_data, contamination_image_data

train_image_data, test_image_data, large_image_data, small_image_data, contamination_image_data = load_data(train_data_dir, test_data_dir, large_data_dir, small_data_dir, contamination_data_dir)


In [4]:
#Segmentation용 이미지를 데이터로 변환
def load_data(gr_tr_large_dir, gr_tr_small_dir, gr_tr_contamination_dir):
    gr_tr_large_data = load_image_data(gr_tr_large_dir)
    gr_tr_small_data = load_image_data(gr_tr_small_dir)
    gr_tr_contamination_data = load_image_data(gr_tr_contamination_dir)
    return gr_tr_large_data, gr_tr_small_data, gr_tr_contamination_data

gr_tr_large_data, gr_tr_small_data, gr_tr_contamination_data = load_data(gr_tr_large_dir, gr_tr_small_dir, gr_tr_contamination_dir)


In [5]:
#배열로 변환, normalization
def preprocess_image_data(image_data):
    return np.array(image_data) / 255

train_image_data = preprocess_image_data(train_image_data)
test_image_data = preprocess_image_data(test_image_data)
small_image_data = preprocess_image_data(small_image_data)
large_image_data = preprocess_image_data(large_image_data)
contamination_image_data = preprocess_image_data(contamination_image_data)
gr_tr_large_data = preprocess_image_data(gr_tr_large_data)
gr_tr_small_data = preprocess_image_data(gr_tr_small_data)
gr_tr_contamination_data = preprocess_image_data(gr_tr_contamination_data)


In [6]:
#데이터 확인
test_image_data[0][16][16]

array([0.40392157, 0.34117648, 0.32156864], dtype=float32)

#데이터 전처리

크게 파손, 적게 파손, 오염된 데이터 셋 모두 augmentation을 통해 데이터의 양을 늘린 뒤에, train과 test로 분리 후에 예측 결과 확인

In [7]:
#배열 변환된 데이터 셋
"""
train_image_data : 정상 훈련 데이터 (200)
test_image_data : 정상 테스트 데이터 (20)
small_image_data : 적게 파손 (20)
large_image_data : 크게 파손 (20)
contamination_image_data : 오염 (20)
"""

'\ntrain_image_data : 정상 훈련 데이터 (200)\ntest_image_data : 정상 테스트 데이터 (20)\nsmall_image_data : 적게 파손 (20)\nlarge_image_data : 크게 파손 (20)\ncontamination_image_data : 오염 (20)\n'

In [8]:
#정상 데이터셋이 너무 과도하게 많기 때문에 50개 정도로 추출
subset_train_data = train_image_data[:50]

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# 이미지 데이터 증강을 위한 ImageDataGenerator 생성
datagen = ImageDataGenerator(
    rotation_range=20,  # 이미지 회전 범위 (0~20도 사이에서 무작위로 회전)
    width_shift_range=0.1,  # 가로로 이미지 이동 범위 (전체 너비의 0.1 비율로 무작위로 이동)
    height_shift_range=0.1,  # 세로로 이미지 이동 범위 (전체 높이의 0.1 비율로 무작위로 이동)
    shear_range=0.2,  # 이미지 변형을 위한 전단 강도 (0.2 비율 내외로 무작위로 전단)
    zoom_range=0.2,  # 이미지 확대/축소 범위 (0.2 비율 내외로 무작위로 확대/축소)
    horizontal_flip=True,  # 이미지 수평 뒤집기
    fill_mode='nearest'  # 이미지 변형 시 채울 픽셀 값 설정
)

# augmented 이미지 개수
desired_num_augmented_images = 10

In [31]:
def augment_images(image_data, desired_num_augmented_images):
    augmented_images = []
    num_generated_images = 0

    while num_generated_images < desired_num_augmented_images:
        image = image_data[0]
        generated_images = datagen.flow(np.expand_dims(image, axis=0), batch_size=1)

        augmented_images.extend(generated_images[0])
        num_generated_images = len(augmented_images)

    augmented_images = np.array(augmented_images[:desired_num_augmented_images])
    return augmented_images

# broken_small 이미지 증강
augmented_small_images = augment_images(small_image_data, desired_num_augmented_images)

# broken_large 이미지 증강
augmented_large_images = augment_images(large_image_data, desired_num_augmented_images)

# contamination 이미지 증강
augmented_contamination_images = augment_images(contamination_image_data, desired_num_augmented_images)


In [32]:
#데이터 개수 확인
print(test_image_data.shape[0])
print(small_image_data.shape[0])
print(large_image_data.shape[0])
print(contamination_image_data.shape[0])
print(augmented_small_images.shape[0])
print(augmented_large_images.shape[0])
print(augmented_contamination_images.shape[0])

20
22
20
21
10
10
10


In [33]:
#각각의 데이터와 augmented_image합친 후 shuffle
combined_small_data = np.concatenate((small_image_data, augmented_small_images), axis=0)
np.random.shuffle(combined_small_data)

combined_large_data = np.concatenate((large_image_data, augmented_large_images), axis=0)
np.random.shuffle(combined_large_data)

combined_contamination_data = np.concatenate((contamination_image_data, augmented_contamination_images), axis=0)
np.random.shuffle(combined_contamination_data)

#적게 파손, 크게 파손 모두 파손된 데이터로 합쳐서 분류
combined_broken_data = np.concatenate((combined_small_data, combined_large_data), axis=0)

In [34]:
#배열 변환된 데이터 셋
"""
subset_train_data : 정상 훈련 데이터
test_image_data : 정상 테스트 데이터
combined_broken_data : augmentation 진행한 broken_small, broken_large data
combined_contamination_data : augmentation 진행한 contamination_data
"""

'\nsubset_train_data : 정상 훈련 데이터\ntest_image_data : 정상 테스트 데이터\ncombined_broken_data : augmentation 진행한 broken_small, broken_large data\ncombined_contamination_data : augmentation 진행한 contamination_data\n'

In [35]:
#Train set 구성

# 데이터 개수 구하기
broken_num_samples = combined_broken_data.shape[0]
contamination_num_samples = combined_contamination_data.shape[0]

# 90%의 데이터 개수 계산
b_90 = int(0.90 * broken_num_samples)
c_90 = int(0.90 * contamination_num_samples)

# 90% 데이터 추출
b_new_data = combined_broken_data[:b_90]
c_new_data = combined_contamination_data[:c_90]

# subset_train_data와 combined_small_data 합치기
resized_images = np.concatenate((subset_train_data, b_new_data, c_new_data), axis=0)

In [36]:
all_labels = []
for i in range(0,subset_train_data.shape[0]):
  all_labels.append([1,0,0])
for j in range(0,b_new_data.shape[0]):
  all_labels.append([0,1,0])
for k in range(0,c_new_data.shape[0]):
  all_labels.append([0,0,1])
all_labels = np.array(all_labels)

In [37]:
#Test set 구성
b_test_data = combined_broken_data[b_90:]
c_test_data = combined_contamination_data[c_90:]
test_set_data = np.concatenate((test_image_data, b_test_data, c_test_data),axis=0)

test_labels = []
for i in range(0,test_image_data.shape[0]):
  test_labels.append([1,0,0])
for j in range(0,b_test_data.shape[0]):
  test_labels.append([0,1,0])
for k in range(0,c_test_data.shape[0]):
  test_labels.append([0,0,1])
test_labels = np.array(test_labels)

In [38]:
#최종 데이터셋 shape 확인
print(resized_images.shape)
print(all_labels.shape)
print(test_set_data.shape)
print(test_labels.shape)

(132, 64, 64, 3)
(132, 3)
(31, 64, 64, 3)
(31, 3)


#모델 구성 및 결과 예측

In [41]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential


# CNN 모델을 함수 형태로 정의
def create_model():
    model = Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(256, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# KerasClassifier를 사용하여 Keras 모델을 래핑
keras_model = KerasClassifier(build_fn=create_model)

# 그리드 탐색을 위한 hyperparameter 조합 설정
param_grid = {
    'batch_size': [32, 64],
    'epochs': [10, 20, 30]
}

# GridSearchCV를 사용하여 그리드 탐색 수행
grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=3)
grid_search_result = grid_search.fit(resized_images, all_labels)

# 최적의 hyperparameter 조합과 결과 출력
print("Best Score: ", grid_search_result.best_score_)
print("Best Params: ", grid_search_result.best_params_)

Epoch 1/10


<ipython-input-41-13f99bc52fcc>:23: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_model = KerasClassifier(build_fn=create_model)


3/3 [==============================] - 3s 621ms/step - loss: 1.0437 - accuracy: 0.4773
Epoch 2/10
3/3 [==============================] - 3s 1s/step - loss: 0.9578 - accuracy: 0.3864
Epoch 3/10
3/3 [==============================] - 2s 581ms/step - loss: 0.8842 - accuracy: 0.6250
Epoch 4/10
3/3 [==============================] - 2s 511ms/step - loss: 0.8604 - accuracy: 0.6250
Epoch 5/10
3/3 [==============================] - 2s 518ms/step - loss: 0.8680 - accuracy: 0.6250
Epoch 6/10
3/3 [==============================] - 2s 515ms/step - loss: 0.8474 - accuracy: 0.6250
Epoch 7/10
3/3 [==============================] - 2s 508ms/step - loss: 0.8381 - accuracy: 0.6250
Epoch 8/10
3/3 [==============================] - 2s 514ms/step - loss: 0.8314 - accuracy: 0.6250
Epoch 9/10
3/3 [==============================] - 2s 725ms/step - loss: 0.8444 - accuracy: 0.6250
Epoch 10/10
2/2 [==============================] - 0s 67ms/step - loss: 2.2681 - accuracy: 0.0000e+00
Epoch 1/10
3/3 [==============

2/2 [==============================] - 0s 75ms/step - loss: 9.1390 - accuracy: 0.3864
Epoch 1/20
3/3 [==============================] - 3s 731ms/step - loss: 1.0577 - accuracy: 0.5000
Epoch 2/20
3/3 [==============================] - 3s 980ms/step - loss: 0.8964 - accuracy: 0.6250
Epoch 3/20
3/3 [==============================] - 2s 560ms/step - loss: 0.8914 - accuracy: 0.6250
Epoch 4/20
3/3 [==============================] - 2s 516ms/step - loss: 0.8322 - accuracy: 0.6250
Epoch 5/20
3/3 [==============================] - 2s 529ms/step - loss: 0.8676 - accuracy: 0.6250
Epoch 6/20
3/3 [==============================] - 2s 518ms/step - loss: 0.8428 - accuracy: 0.6250
Epoch 7/20
3/3 [==============================] - 2s 514ms/step - loss: 0.8184 - accuracy: 0.6250
Epoch 8/20
3/3 [==============================] - 2s 526ms/step - loss: 0.8522 - accuracy: 0.6250
Epoch 9/20
3/3 [==============================] - 2s 812ms/step - loss: 0.8131 - accuracy: 0.6250
Epoch 10/20
3/3 [===============

2/2 [==============================] - 0s 68ms/step - loss: 1.9168 - accuracy: 0.0000e+00
Epoch 1/20
3/3 [==============================] - 3s 531ms/step - loss: 1.0641 - accuracy: 0.3295
Epoch 2/20
3/3 [==============================] - 2s 530ms/step - loss: 1.0334 - accuracy: 0.5114
Epoch 3/20
3/3 [==============================] - 2s 552ms/step - loss: 1.0283 - accuracy: 0.5000
Epoch 4/20
3/3 [==============================] - 3s 980ms/step - loss: 0.9836 - accuracy: 0.5000
Epoch 5/20
3/3 [==============================] - 2s 733ms/step - loss: 0.9563 - accuracy: 0.5114
Epoch 6/20
3/3 [==============================] - 2s 524ms/step - loss: 0.8575 - accuracy: 0.6477
Epoch 7/20
3/3 [==============================] - 2s 527ms/step - loss: 0.7868 - accuracy: 0.7386
Epoch 8/20
3/3 [==============================] - 2s 525ms/step - loss: 0.7644 - accuracy: 0.6932
Epoch 9/20
3/3 [==============================] - 2s 517ms/step - loss: 0.7477 - accuracy: 0.5909
Epoch 10/20
3/3 [===========

Best Params:  {'batch_size': 64, 'epochs': 30} 으로 확인했지만 accuracy 값을 봤을 때 과적합이 발생했을 확률이 굉장히 높다고 판단되기 떄문에 {'batch_size': 64, 'epochs': 15}로 모델학습

In [43]:
model = Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#모델 학습
model.fit(resized_images, all_labels, epochs=15, batch_size=64, verbose=1)

Epoch 1/15
3/3 [==============================] - 4s 687ms/step - loss: 1.1243 - accuracy: 0.3409
Epoch 2/15
3/3 [==============================] - 2s 648ms/step - loss: 1.1087 - accuracy: 0.2652
Epoch 3/15
3/3 [==============================] - 3s 660ms/step - loss: 1.0818 - accuracy: 0.4167
Epoch 4/15
3/3 [==============================] - 3s 827ms/step - loss: 1.0528 - accuracy: 0.6288
Epoch 5/15
3/3 [==============================] - 4s 1s/step - loss: 1.1343 - accuracy: 0.4167
Epoch 6/15
3/3 [==============================] - 2s 648ms/step - loss: 1.0486 - accuracy: 0.4167
Epoch 7/15
3/3 [==============================] - 2s 650ms/step - loss: 1.0664 - accuracy: 0.4167
Epoch 8/15
3/3 [==============================] - 3s 653ms/step - loss: 1.0763 - accuracy: 0.4167
Epoch 9/15
3/3 [==============================] - 3s 747ms/step - loss: 1.0672 - accuracy: 0.4167
Epoch 10/15
3/3 [==============================] - 5s 1s/step - loss: 1.0548 - accuracy: 0.4167
Epoch 11/15
3/3 [========

In [44]:
# test set에 대한 예측 수행
predictions = model.predict(test_set_data)

# 최종 예측 레이블 생성
predicted_labels = []
for prediction in predictions:
    if prediction[0] > prediction[1] and prediction[0] > prediction[2]:
        predicted_labels.append([1, 0, 0])
    elif prediction[1] > prediction[0] and prediction[1] > prediction[2]:
        predicted_labels.append([0, 1, 0])
    else:
        predicted_labels.append([0, 0, 1])
predicted_labels = np.array(predicted_labels)

#예측률 확인
count = 0
for i in range(0,predicted_labels.shape[0]):
  if np.array_equal(predicted_labels[i], test_labels[i]):
        count += 1

prediction_accuracy = count / predicted_labels.shape[0] * 100
print('test set에 대한 예측률: {}%'.format(prediction_accuracy))


1/1 [==============================] - 0s 236ms/step
test set에 대한 예측률: 87.09677419354838%


In [45]:
#원본 데이터에 대한 예측 및 평가 함수
def predict_and_evaluate(model, data, target_labels):
    predictions = model.predict(data)

    predicted_labels = []
    for prediction in predictions:
        if prediction[0] > prediction[1] and prediction[0] > prediction[2]:
            predicted_labels.append([1, 0, 0])
        elif prediction[1] > prediction[0] and prediction[1] > prediction[2]:
            predicted_labels.append([0, 1, 0])
        else:
            predicted_labels.append([0, 0, 1])
    predicted_labels = np.array(predicted_labels)

    count = 0
    for i in range(predicted_labels.shape[0]):
        if np.array_equal(predicted_labels[i], target_labels):
            count += 1

    prediction_accuracy = count / predicted_labels.shape[0] * 100
    return prediction_accuracy

In [46]:
#원본 데이터에 대한 예측 및 정확도 평가

# broken_large
accuracy_large = predict_and_evaluate(model, large_image_data, [0, 1, 0])
print('broken_large에 대한 예측률: {}%'.format(accuracy_large))

# broken_small
accuracy_small = predict_and_evaluate(model, small_image_data, [0, 1, 0])
print('broken_small에 대한 예측률: {}%'.format(accuracy_small))

#contamination
accuracy_contamination = predict_and_evaluate(model, contamination_image_data, [0,0,1])
print('contamination에 대한 예측률:{}%'.format(accuracy_contamination))

1/1 [==============================] - 0s 115ms/step
broken_large에 대한 예측률: 100.0%
1/1 [==============================] - 0s 128ms/step
broken_small에 대한 예측률: 77.27272727272727%
1/1 [==============================] - 0s 126ms/step
contamination에 대한 예측률:0.0%


#Image Segmentation


In [11]:
!pip install -q git+https://github.com/tensorflow/examples.git

  Preparing metadata (setup.py) ... done


In [31]:
# U-Net model
# coded by st.watermelon
#https://pasus.tistory.com/204 링
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Activation, BatchNormalization, Concatenate


In [ ]:
#배열 변환된 ground_truth 이미지
"""
gr_tr_large_data : ground_truth 데이터 (20)
gr_tr_small_data : ground_truth 데이터 (22)
gr_tr_contamination_data : ground_truth 데이터 (21)
large_image_data : 크게 파손 (20)
small_image_data : 적게 파손 (22)
contamination_image_data : 오염 (21)
"""

In [33]:
#train, test set 분리
gr_tr_large_80,gr_tr_small_80, gr_tr_conta_80  = 16, 17, 17
gr_tr_train_set = np.concatenate((gr_tr_large_data[:gr_tr_large_80], gr_tr_small_data[:gr_tr_small_80], gr_tr_contamination_data[:gr_tr_conta_80]),axis=0)
gr_tr_test_set = np.concatenate((gr_tr_large_data[gr_tr_large_80:], gr_tr_small_data[gr_tr_small_80:], gr_tr_contamination_data[gr_tr_conta_80:]),axis=0)
train_image_set = np.concatenate((large_image_data[:gr_tr_large_80], small_image_data[:gr_tr_small_80], contamination_image_data[:gr_tr_conta_80]),axis=0)
test_image_set = np.concatenate((large_image_data[gr_tr_large_80:], small_image_data[gr_tr_small_80:], contamination_image_data[gr_tr_conta_80:]),axis=0)

In [34]:
print(gr_tr_train_set.shape, gr_tr_test_set.shape, train_image_set.shape, test_image_set.shape)

(50, 64, 64, 3) (13, 64, 64, 3) (50, 64, 64, 3) (13, 64, 64, 3)


In [ ]:
#Train, Test set
"""
gr_tr_train_set, train_image_set : train (50)
gr_tr_test_set, test_image_set : test (13)
"""

In [ ]:
#이미지, ground_truth 확인
def plot_image(image):
    plt.imshow(image)
    plt.axis('off')
    plt.show()
# gr_tr_large_data와 large_image_data 이미지 동시 plot
plot_image(gr_tr_large_data[0])
plot_image(large_image_data[0])

In [43]:

""" Conv Block """
class ConvBlock(tf.keras.layers.Layer):
    def __init__(self, n_filters):
        super(ConvBlock, self).__init__()

        self.conv1 = Conv2D(n_filters, 3, padding='same')
        self.conv2 = Conv2D(n_filters, 3, padding='same')

        self.bn1 = BatchNormalization()
        self.bn2 = BatchNormalization()

        self.activation = Activation('relu')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.activation(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.activation(x)

        return x


""" Encoder Block """
class EncoderBlock(tf.keras.layers.Layer):
    def __init__(self, n_filters):
        super(EncoderBlock, self).__init__()

        self.conv_blk = ConvBlock(n_filters)
        self.pool = MaxPooling2D((2,2))

    def call(self, inputs):
        x = self.conv_blk(inputs)
        p = self.pool(x)
        return x, p


""" Decoder Block """
class DecoderBlock(tf.keras.layers.Layer):
    def __init__(self, n_filters):
        super(DecoderBlock, self).__init__()

        self.up = Conv2DTranspose(n_filters, (2,2), strides=2, padding='same')
        self.conv_blk = ConvBlock(n_filters)

    def call(self, inputs, skip):
        x = self.up(inputs)
        x = Concatenate()([x, skip])
        x = self.conv_blk(x)

        return x


""" U-Net Model """
class UNET(tf.keras.Model):
    def __init__(self, n_classes):
        super(UNET, self).__init__()

        # Encoder
        self.e1 = EncoderBlock(64)
        self.e2 = EncoderBlock(128)
        self.e3 = EncoderBlock(256)
        self.e4 = EncoderBlock(512)

        # Bridge
        self.b = ConvBlock(1024)

        # Decoder
        self.d1 = DecoderBlock(512)
        self.d2 = DecoderBlock(256)
        self.d3 = DecoderBlock(128)
        self.d4 = DecoderBlock(64)

        # Outputs
        if n_classes == 1:
            activation = 'sigmoid'
        else:
            activation = 'softmax'

        self.outputs = Conv2D(n_classes, 1, padding='same', activation=activation)

    def call(self, inputs):
        s1, p1 = self.e1(inputs)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)

        b = self.b(p4)

        d1 = self.d1(b, s4)
        d2 = self.d2(d1, s3)
        d3 = self.d3(d2, s2)
        d4 = self.d4(d3, s1)

        outputs = self.outputs(d4)

        return outputs

In [44]:
# U-Net Model
class UNET(tf.keras.Model):
    # U-Net 모델 구현 (이전 설명의 코드 활용)

  # 학습 데이터와 검증 데이터로 분할
  train_data = (train_image_set, gr_tr_train_set)
  val_data = (test_image_set, gr_tr_test_set)

  # 모델 인스턴스화
  model = UNET(n_classes)

  # 손실 함수 및 최적화 알고리즘 설정
  loss_fn = tf.keras.losses.CategoricalCrossentropy()
  optimizer = tf.keras.optimizers.Adam()

  # 메트릭 설정 (선택 사항)
  metrics = [tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.MeanIoU(n_classes)]

  # 모델 컴파일
  model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

  # 모델 학습
  epochs = 15
  batch_size = 16
  model.fit(train_data[0], train_data[1], validation_data=(val_data[0], val_data[1]), batch_size=batch_size, epochs=epochs)


  # 모델 평가
  loss, accuracy, mean_iou = model.evaluate(test_image_set, gr_tr_test_set)
  print("Test Loss:", loss)
  print("Test Accuracy:", accuracy)
  print("Test Mean IoU:", mean_iou)

Epoch 1/15
4/4 [==============================] - 16s 325ms/step - loss: 0.2901 - categorical_accuracy: 0.0292 - mean_io_u_8: 0.4616 - val_loss: 0.2425 - val_categorical_accuracy: 0.1164 - val_mean_io_u_8: 0.4632
Epoch 2/15
4/4 [==============================] - 0s 101ms/step - loss: 0.2708 - categorical_accuracy: 0.0055 - mean_io_u_8: 0.4616 - val_loss: 0.2426 - val_categorical_accuracy: 0.0293 - val_mean_io_u_8: 0.4632
Epoch 3/15
4/4 [==============================] - 0s 90ms/step - loss: 0.2712 - categorical_accuracy: 0.0013 - mean_io_u_8: 0.4616 - val_loss: 0.2426 - val_categorical_accuracy: 0.1202 - val_mean_io_u_8: 0.4632
Epoch 4/15
4/4 [==============================] - 0s 105ms/step - loss: 0.2621 - categorical_accuracy: 0.0093 - mean_io_u_8: 0.4616 - val_loss: 0.2463 - val_categorical_accuracy: 0.5392 - val_mean_io_u_8: 0.4632
Epoch 5/15
4/4 [==============================] - 0s 92ms/step - loss: 0.2632 - categorical_accuracy: 0.0029 - mean_io_u_8: 0.4616 - val_loss: 0.9219 - 